In [5]:
from app import crud, models, schemas
from sqlalchemy.sql import func
from app.services.controlling import ControllingCalendar

In [6]:
without_discharge = crud.claim.get_query(db).filter(models.Claim.discharge.is_(None))

NameError: name 'db' is not defined

In [7]:
from app.db.session import SessionLocal

In [8]:
db = SessionLocal()

In [ ]:
fn = without_discharge.filter(models.Claim.store_internal_id==205)

In [9]:
calendar = ControllingCalendar(year=2020, monthrange=[1,12]).get()
start = calendar['range']['start']
end = calendar['range']['end']

In [ ]:
fried = crud.store.get_by_kwargs(db, internal_id=205).first()

In [ ]:
fried.claims.filter(models.Claim.discharge.is_(None)).with_entities(func.sum(models.Claim.bill).label('bill')).first()

In [ ]:
bill = fried.claims.filter(models.Claim.created_at.between(start, end)).with_entities(func.sum(models.Claim.bill).label('bill')).first()

In [ ]:
bill

In [10]:
from sqlalchemy.orm import Session
from datetime import datetime

In [11]:
def get_clean_discharge(
        _db: Session = db, *, start: datetime = None, end: datetime = None,
        store_internal_id: int = None, owner_id: int = None
) -> float:
    """
    TO AVOID MULTIPLE COUNTED DISCHARGES
    WE ONLY ACCEPT ONE DISCHARGE PER CONTRACT
    MEANS WE VE TO EXCLUDE ALL OTHER ROWS WITH THE SAME CONTRACT NR
    """
    exclude_contracts: t.List[int] = []
    discharges: t.List[float] = []
    #  FOR BETTER SQL PERFORMANCE WE ENCLOSE OUR QUERY: EXCLUDE ALL WITH DISCHARGE IS NULL;
    claims = crud.claim.get_query(db)
    if start and end:
        claims = claims.filter(models.Claim.created_at.between(start, end))  # noqa
    if store_internal_id:
        claims = claims.filter(models.Claim.store_internal_id == store_internal_id)  # noqa
    if owner_id:
        claims = claims.filter(models.Claim.owner_id == owner_id)  # noqa  
    claims = claims.filter(models.Claim.discharge.isnot(None))  # noqa 
    claims: t.List[models.Claim] = claims.all()  # noqa

    for claim in claims:
        if claim.contract_nr not in exclude_contracts:
            exclude_contracts.append(claim.contract_nr)
            discharges.append(claim.discharge)
    total_discharges: float = sum(discharges)
    total_discharges_round: float = round(total_discharges, 2)
    return total_discharges_round

In [22]:
x = get_clean_discharge(start=start, end=end, store_internal_id=188, owner_id=5)

In [23]:
x

-3679.37